In [1]:
import os
import random
import time
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import scipy.io as sio
from sklearn.datasets import load_svmlight_files
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import pandas as pd
from sklearn.neural_network import MLPClassifier

In [2]:
def readPIE(path):
    root = 'D:\pytorch\统计学习方法实践\PIE'
    fin_path = os.path.join(root, path)
    data = sio.loadmat(fin_path)
    X,y = data['fea'].astype(np.float64),data['gnd'].ravel()
    y = LabelEncoder().fit(y).transform(y).astype(int)
    return X,y


def readBooks(path):
    root = 'D:\pytorch\统计学习方法实践\Amazon Reviews'
    file = os.path.join(root, path)
    X, y = load_svmlight_files([file])
    X = X.toarray()
    y = LabelEncoder().fit(y).transform(y).astype(np.float64)
    #X = scale(X)
    return X,y

In [3]:
def logistic(x):
    return 1 / (1 + np.exp(-x))


def logistic_derivative(x):
    return logistic(x) * (1 - logistic(x))


class ANN:
    def predict(self, x):
        for w, b in zip(self.weights, self.biases):
            z = np.dot(x, w) + b
            x = self.activation(z)
        return self.classes_[np.argmax(x, axis=1)]


class BP(ANN):
    def __init__(self, layers, batch):
        self.layers = layers
        self.num_layers = len(layers)
        self.batch = batch
        self.activation = logistic
        self.activation_deactivation = logistic_derivative
        self.biases = [np.random.rand(x) for x in layers[1:]]
        self.weights = [np.random.rand(x, y) for x, y in zip(layers[: -1], layers[1:])]


    def fit(self, x, y, lr, epochs):
        label_bin = LabelBinarizer()
        y = label_bin.fit_transform(y)
        self.classes_ = label_bin.classes_
        train_data = [(x, y) for x, y in zip(x, y)]
        n = len(train_data)
        for i in range(epochs):
            random.shuffle(train_data)
            batches = [train_data[k: k + self.batch] for k in range(0, n, self.batch)]
            for sub_batch in batches:
                sub_x = []
                sub_y = []
                for tmp_x, tmp_y in sub_batch:
                    sub_x.append(tmp_x)
                    sub_y.append(tmp_y)
                activations = [np.array(sub_x)]
                for w, b in zip(self.weights, self.biases):
                    res = np.dot(activations[-1], w) + b
                    output = self.activation(res)
                    activations.append(output)
                err = activations[-1] - np.array(sub_y)
                details = [err * self.activation_deactivation(activations[-1])]
                for i in range(self.num_layers - 2, 0, -1):
                    details.append(self.activation_deactivation(activations[i]) *
                                    np.dot(details[-1], self.weights[i].T))
                details.reverse()
                for j in range(self.num_layers - 1):
                    details = lr / self.batch * ((np.atleast_2d(activations[j].sum(axis=0)).T).dot(np.atleast_2d(
                        details[j].sum(axis=0)
                    )))
                    self.weights[j] -= details
                    details = lr / self.batch * details[j].sum(axis=0)
                    self.biases[j] -= details
        return self

In [4]:
def PIE_eval(X, y, res, weights):
    X_t, y_t = X, y
    params = {
        'booster': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 5,
        'min_data_in_leaf': 91,
        'max_bin': 205,
        'max_depth': 8,
        'num_leaves': 20,
        'max_bin': 50,
        "learning_rate": 0.01,
        'feature_fraction': 0.6,
        "bagging_fraction": 1.0,  # 每次迭代时用的数据比例
        'bagging_freq': 45,
        'min_split_gain': 1.0,
        'min_child_samples': 10,
        'lambda_l1': 0.3, 'lambda_l2': 0.6,
        'n_jobs': -1,
        'silent': True,  # 信息输出设置成1则没有信息输出
        'seed': 1000,
        'verbose': -1,
    }
    for i in range(len(X_t)):
        if weights:
            # org_tr_step, tr_step = len(X_t[i]) // len(set(y_t[i])), len(X_t[i]) // len(set(y_t[i])) // 2
            # new_X_train, new_y_train = X_t[i][:tr_step,:], y_t[i][:tr_step]
            #
            # for f in range(org_tr_step, len(X_t[i]), org_tr_step):
            #     new_X_train = np.vstack((new_X_train, X_t[i][f:f+tr_step,:]))
            #     new_y_train = np.hstack((new_y_train, y_t[i][f:f+tr_step]))

            for j in range(len(X_t)):
                X_test, y_test = X_t[j], y_t[j]

                org_tt_step, tt_step = len(X_t[j]) // len(set(y_t[j])), int(len(X_t[j]) // len(set(y_t[j])) * weights)
                tmp_nd = X_t[j][: org_tt_step, :]
                random.shuffle(tmp_nd)

                new_X_test = tmp_nd[: tt_step, :]
                real_X_test = tmp_nd[tt_step: org_tt_step, :]

                new_y_test = y_t[j][:tt_step]
                real_y_test = y_t[j][tt_step: org_tt_step]

                for f in range(org_tt_step, len(X_t[j]), org_tt_step):
                    tmp_nd = X_t[j][f: f + org_tt_step, :]
                    random.shuffle(tmp_nd)

                    new_X_test = np.vstack((new_X_test, tmp_nd[: tt_step, :]))
                    real_X_test = np.vstack((real_X_test, tmp_nd[tt_step: org_tt_step, :]))

                    new_y_test = np.hstack((new_y_test, y_t[j][f: f + tt_step]))
                    real_y_test = np.hstack((real_y_test, y_t[j][f + tt_step: f + org_tt_step]))

                fin_X_train, fin_y_train = np.vstack((X_t[i], new_X_test)), np.hstack((y_t[i], new_y_test))

                train_test = np.vstack([fin_X_train, real_X_test])  # new training data
                lgb_data = lgb.Dataset(train_test, label=np.array([0] * len(fin_X_train) + [1] * len(real_X_test)))
                result = lgb.cv(params, lgb_data, num_boost_round=100, nfold=10, verbose_eval=20)

                clf = BP([fin_X_train.shape[1], 68], 100).fit(x=fin_X_train, y=fin_y_train, lr=1e-1, epochs=3000)
                predict_label = clf.predict(real_X_test)

                dic = classification_report(predict_label, real_y_test, output_dict=True)
                res[i].append((dic['accuracy'], min(result['auc-mean'])))
                print(classification_report(predict_label, y_test))
        else:
            X_train, y_train = X_t[i], y_t[i]
            clf = BP([X_train.shape[1], 68], 100).fit(x=X_train, y=y_train, lr=1e-1, epochs=3000)
            for j in range(len(X_t)):
                X_test, y_test = X_t[j], y_t[j]

                train_test = np.vstack([X_train, X_test])  # new training data
                lgb_data = lgb.Dataset(train_test, label=np.array([0] * len(X_train) + [1] * len(X_test)))
                result = lgb.cv(params, lgb_data, num_boost_round=100, nfold=10, verbose_eval=20)

                predict_label = clf.predict(X_test)

                dic = classification_report(predict_label, y_test, output_dict=True)
                res[i].append((dic['accuracy'], min(result['auc-mean'])))
                print(classification_report(predict_label, y_test))

def Book_eval(X, y, res, weights):
    X_t, y_t = X, y
    params = {
        'booster': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 5,
        'min_data_in_leaf': 91,
        'max_bin': 205,
        'max_depth': 8,
        'num_leaves': 20,
        'max_bin': 50,
        "learning_rate": 0.01,
        'feature_fraction': 0.6,
        "bagging_fraction": 1.0,  # 每次迭代时用的数据比例
        'bagging_freq': 45,
        'min_split_gain': 1.0,
        'min_child_samples': 10,
        'lambda_l1': 0.3, 'lambda_l2': 0.6,
        'n_jobs': -1,
        'silent': True,  # 信息输出设置成1则没有信息输出
        'seed': 1000,
        'verbose': -1,
    }
    for i in range(len(X_t)):
        if weights:
            line = int(1000 * weights)
            for j in range(len(X_t)):
                X_test, y_test = X_t[j], y_t[j]
                tmp_adn1 = X_test[:len(X_test)//2, :]
                tmp_adn2 = X_test[len(X_test//2): len(X_test), :]
                random.shuffle(tmp_adn1)
                random.shuffle(tmp_adn2)

                new_X_test, new_y_test = np.vstack((tmp_adn1[:len(tmp_adn1)//2, :], tmp_adn2[:len(tmp_adn2)//2, :])), \
                                        np.hstack((y_test[:len(X_t[j])//4], y_test[len(X_t[j])//2: len(X_t[j])*3//4]))
                real_X_test, real_y_test = np.vstack((tmp_adn1[len(tmp_adn1)//2:len(tmp_adn1), :], tmp_adn2[len(tmp_adn2)//2:len(tmp_adn2), :])), \
                                        np.hstack((y_test[len(X_t[j])//4:len(X_t[j])//2], y_test[len(X_t[j])*3//4:]))

                fin_X_train, fin_y_train = np.vstack((X_t[i], new_X_test)), np.hstack((y_t[i], new_y_test))

                train_test = np.vstack([fin_X_train, real_X_test])  # new training data
                lgb_data = lgb.Dataset(train_test, label=np.array([0] * len(fin_X_train) + [1] * len(real_X_test)))
                result = lgb.cv(params, lgb_data, num_boost_round=100, nfold=10, verbose_eval=20)

                clf = BernoulliNB().fit(fin_X_train, fin_y_train)
                predict_label = clf.predict(real_y_test)

                dic = classification_report(predict_label, y_test, output_dict=True)
                res[i].append((dic['accuracy'], min(result['auc-mean'])))
                print(classification_report(predict_label, y_test))
        else:
            X_train, y_train = X_t[i], y_t[i]
            clf = BernoulliNB().fit(X_train, y_train)
            for j in range(len(X_t)):
                X_test, y_test = X_t[j], y_t[j]

                train_test = np.vstack([X_train, X_test])  # new training data
                lgb_data = lgb.Dataset(train_test, label=np.array([0] * len(X_train) + [1] * len(X_test)))
                result = lgb.cv(params, lgb_data, num_boost_round=100, nfold=10, verbose_eval=20)

                predict_label = clf.predict(X_test)

                dic = classification_report(predict_label, y_test, output_dict=True)
                res[i].append((dic['accuracy'], min(result['auc-mean'])))
                print(classification_report(predict_label, y_test))

In [5]:
PIE_path = ['C05.mat', 'C07.mat', 'C09.mat', 'C27.mat', 'C29.mat']
Book_path = ['books.svmlight', 'dvd.svmlight', 'electronics.svmlight', 'kitchen.svmlight']

X_t, y_t = [], []
X_t_b, y_t_b = [], []
for i in PIE_path:
    xtmp, ytmp = readPIE(i)
    X_t.append(xtmp)
    y_t.append(ytmp)
for i in Book_path:
    xtmp, ytmp = readBooks(i)
    X_t_b.append(xtmp)
    y_t_b.append(ytmp)

In [6]:
X_train, X_test, y_train, y_test = X_t[0], X_t[1], y_t[0], y_t[1]
clf = BP([X_train.shape[1], 68], 100).fit(X_train, y_train, lr=1e-1, epochs=3000)
pl = clf.predict(X_test)
print(classification_report(pl, y_test))

              precision    recall  f1-score   support

           0       0.46      0.41      0.43        27
           1       0.04      0.20      0.07         5
           2       0.29      0.33      0.31        21
           3       0.00      0.00      0.00         6
           4       0.33      0.50      0.40        16
           5       0.79      0.43      0.56        44
           6       0.00      0.00      0.00         0
           7       0.38      0.39      0.38        23
           8       0.00      0.00      0.00         6
           9       0.83      1.00      0.91        20
          10       0.08      1.00      0.15         2
          11       0.04      0.04      0.04        26
          12       0.58      0.26      0.36        53
          13       0.00      0.00      0.00         0
          14       0.75      0.06      0.12       288
          15       0.00      0.00      0.00         0
          16       0.92      0.26      0.40        85
          17       0.17    

C:\Users\zk1307403902\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zk1307403902\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zk1307403902\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
clf = MLPClassifier(hidden_layer_sizes=(100, 50), solver='sgd', max_iter=3000)
clf.fit(X_train, y_train)
pl = clf.predict(X_test)
print(classification_report(pl, y_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.00      0.00      0.00         0
           2       0.33      0.67      0.44        12
           3       0.00      0.00      0.00         1
           4       0.58      0.48      0.53        29
           5       0.83      0.80      0.82        25
           6       0.00      0.00      0.00         0
           7       0.04      1.00      0.08         1
           8       0.00      0.00      0.00         3
           9       0.79      1.00      0.88        19
          10       0.00      0.00      0.00         0
          11       0.25      0.17      0.20        35
          12       0.04      0.33      0.07         3
          13       0.00      0.00      0.00         0
          14       0.54      0.07      0.12       198
          15       0.00      0.00      0.00         0
          16       0.21      0.11      0.14        45
          17       0.00    

C:\Users\zk1307403902\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zk1307403902\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zk1307403902\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
train_test = np.vstack([X_train, X_test]) # new training data
lgb_data = lgb.Dataset(train_test, label=np.array([0]*len(X_train)+[1]*len(X_test)))
params = {
        'booster': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 5,
        'min_data_in_leaf': 91,
        'max_bin': 205,
        'max_depth': 8,
        'num_leaves':20,
        'max_bin':50,
        "learning_rate": 0.01,
        'feature_fraction': 0.6,
        "bagging_fraction": 1.0,  # 每次迭代时用的数据比例
        'bagging_freq': 45,
        'min_split_gain': 1.0,
        'min_child_samples': 10,
        'lambda_l1': 0.3, 'lambda_l2': 0.6,
        'n_jobs': -1,
        'silent': True,  # 信息输出设置成1则没有信息输出
        'seed': 1000,
        'verbose': -1,
    }
result = lgb.cv(params, lgb_data, num_boost_round=1000, nfold=10, verbose_eval=20)
print('交叉验证中最优的AUC为 {:.5f}.'.format(min(result['auc-mean'])))

C:\Users\zk1307403902\anaconda3\envs\pytorch\lib\site-packages\lightgbm\basic.py:1222: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  _log_warning('{0} keyword has been found in `params` and will be ignored.\n'


[LightGBM] [Warning] Unknown parameter: booster
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] min_data_in_leaf is set=91, min_child_samples=10 will be ignored. Current value: min_data_in_leaf=91
[20]	cv_agg's auc: 0.997246 + 0.00146789
[40]	cv_agg's auc: 0.997592 + 0.00151323
[60]	cv_agg's auc: 0.997866 + 0.00148784
[80]	cv_agg's auc: 0.998221 + 0.00139575
[100]	cv_agg's auc: 0.998725 + 0.0010156
[120]	cv_agg's auc: 0.999075 + 0.000804294
[140]	cv_agg's auc: 0.999255 + 0.000683637
[160]	cv_agg's auc: 0.999384 + 0.000602501
[180]	cv_agg's auc: 0.999517 + 0.000488133
[200]	cv_agg's auc: 0.999596 + 0.000431784
[220]	cv_agg's auc: 0.999676 + 0.000348516
[240]	cv_agg's auc: 0.999759 + 0.000273996
[260]	cv_agg's auc: 0.999807 + 0.000224682
[280]	cv_agg's auc: 0.999867 + 0.000160139
[300]	cv_agg's auc: 0.999904 + 0.000124931
[320]	cv_agg's auc: 0.999932 + 8.50922e-05
[340]	cv_agg's auc: 0.99995 + 6.26467e-05
[360]	cv_agg's auc: 0.999961 + 5.35371e-05
[380]	cv_agg's auc: 

In [10]:
start = time.time()
PIE_res = [[], [], [], [], []]
PIE_res_w = [[], [], [], [], []]
Book_res = [[], [], [], []]
Book_res_w = [[], [], [], []]

PIE_eval(X_t, y_t, PIE_res, weights=0)
PIE_eval(X_t, y_t, PIE_res_w, weights=0.7)
Book_eval(X_t_b, y_t_b, Book_res, weights=False)
Book_eval(X_t_b, y_t_b, Book_res_w, weights=True)


end = time.time()
print(end - start)

C:\Users\zk1307403902\anaconda3\envs\pytorch\lib\site-packages\lightgbm\basic.py:1222: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  _log_warning('{0} keyword has been found in `params` and will be ignored.\n'


[20]	cv_agg's auc: 0.395652 + 0.0492578
[40]	cv_agg's auc: 0.395551 + 0.0487605
[60]	cv_agg's auc: 0.395431 + 0.0488967
[80]	cv_agg's auc: 0.395431 + 0.0488967
[100]	cv_agg's auc: 0.395431 + 0.0488967
              precision    recall  f1-score   support

           0       0.96      0.98      0.97        48
           1       1.00      0.96      0.98        51
           2       1.00      1.00      1.00        49
           3       1.00      1.00      1.00        49
           4       1.00      0.96      0.98        51
           5       1.00      1.00      1.00        49
           6       0.98      0.98      0.98        49
           7       0.98      1.00      0.99        48
           8       0.98      0.94      0.96        51
           9       1.00      0.98      0.99        50
          10       1.00      0.98      0.99        50
          11       1.00      0.96      0.98        51
          12       0.96      1.00      0.98        47
          13       1.00      0.98      0.9

C:\Users\zk1307403902\anaconda3\envs\pytorch\lib\site-packages\lightgbm\basic.py:1222: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  _log_warning('{0} keyword has been found in `params` and will be ignored.\n'


[20]	cv_agg's auc: 0.997246 + 0.00146789
[40]	cv_agg's auc: 0.997592 + 0.00151323
[60]	cv_agg's auc: 0.997866 + 0.00148784
[80]	cv_agg's auc: 0.998221 + 0.00139575
[100]	cv_agg's auc: 0.998725 + 0.0010156
              precision    recall  f1-score   support

           0       0.42      0.30      0.35        33
           1       0.00      0.00      0.00         0
           2       0.04      1.00      0.08         1
           3       0.00      0.00      0.00         7
           4       0.29      0.23      0.26        30
           5       0.75      0.60      0.67        30
           6       0.21      1.00      0.34         5
           7       0.25      0.60      0.35        10
           8       0.00      0.00      0.00         6
           9       0.92      1.00      0.96        22
          10       0.12      1.00      0.22         3
          11       0.46      0.11      0.18       100
          12       0.29      0.78      0.42         9
          13       0.00      0.00     

C:\Users\zk1307403902\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zk1307403902\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zk1307403902\anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\zk13074039

...

[20]	cv_agg's auc: 0.705466 + 0.0240683
[40]	cv_agg's auc: 0.708161 + 0.0229398
[60]	cv_agg's auc: 0.708561 + 0.0236766
[80]	cv_agg's auc: 0.708373 + 0.0237723
[100]	cv_agg's auc: 0.707722 + 0.0246102
              precision    recall  f1-score   support

         0.0       0.87      0.88      0.88       981
         1.0       0.89      0.87      0.88      1019

    accuracy                           0.88      2000
   macro avg       0.88      0.88      0.88      2000
weighted avg       0.88      0.88      0.88      2000



C:\Users\zk1307403902\anaconda3\envs\pytorch\lib\site-packages\lightgbm\basic.py:1222: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  _log_warning('{0} keyword has been found in `params` and will be ignored.\n'


[20]	cv_agg's auc: 0.648725 + 0.0218311
[40]	cv_agg's auc: 0.651736 + 0.0212567
[60]	cv_agg's auc: 0.654974 + 0.0200197
[80]	cv_agg's auc: 0.656389 + 0.0200869
[100]	cv_agg's auc: 0.656264 + 0.0210158
              precision    recall  f1-score   support

         0.0       0.88      0.92      0.90       960
         1.0       0.92      0.88      0.90      1040

    accuracy                           0.90      2000
   macro avg       0.90      0.90      0.90      2000
weighted avg       0.90      0.90      0.90      2000



C:\Users\zk1307403902\anaconda3\envs\pytorch\lib\site-packages\lightgbm\basic.py:1222: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  _log_warning('{0} keyword has been found in `params` and will be ignored.\n'


[20]	cv_agg's auc: 0.432116 + 0.030088
[40]	cv_agg's auc: 0.429395 + 0.0270404
[60]	cv_agg's auc: 0.426982 + 0.0245349
[80]	cv_agg's auc: 0.425084 + 0.0229998
[100]	cv_agg's auc: 0.421209 + 0.0207592
              precision    recall  f1-score   support

         0.0       0.91      0.94      0.92       972
         1.0       0.94      0.91      0.93      1028

    accuracy                           0.92      2000
   macro avg       0.92      0.92      0.92      2000
weighted avg       0.92      0.92      0.92      2000

                                           1-Other  \
ACC&AUC    (0.984093637454982, 0.3950802180504937)   
ACC&AUC   (0.21578631452581032, 0.990132916980755)   
ACC&AUC  (0.18817527010804322, 0.9792619625384926)   
ACC&AUC   (0.4042617046818727, 0.9907439378796934)   
ACC&AUC  (0.21218487394957983, 0.9980615794561674)   

                                          2-Other  \
ACC&AUC   (0.285451197053407, 0.9897927762265037)   
ACC&AUC  (0.9815837937384899, 0.3560939161

In [12]:
# 显示所有列
pd.set_option('display.max_columns', None)
# 显示所有行
pd.set_option('display.max_rows', None)

print('****************************************************************')
PIE_df = pd.DataFrame(PIE_res, columns=['1-Other', '2-Other', '3-Other', '4-Other', '5-Other'],
                    index=['ACC&AUC' for _ in range(5)])
print(PIE_df)
print('****************************************************************')
PIE_df_w = pd.DataFrame(PIE_res_w, columns=['1-Other', '2-Other', '3-Other', '4-Other', '5-Other'],
                    index=['ACC&AUC' for _ in range(5)])
print(PIE_df_w)
print('****************************************************************')
Book_df = pd.DataFrame(Book_res, columns=['1-Other', '2-Other', '3-Other', '4-Other'],
                    index=['ACC&AUC' for _ in range(4)])
print(Book_df)
print('****************************************************************')
Book_df_w = pd.DataFrame(Book_res_w, columns=['1-Other', '2-Other', '3-Other', '4-Other'],
                    index=['ACC&AUC' for _ in range(4)])
print(Book_df_w)

****************************************************************
                                           1-Other  \
ACC&AUC    (0.984093637454982, 0.3950802180504937)   
ACC&AUC   (0.21578631452581032, 0.990132916980755)   
ACC&AUC  (0.18817527010804322, 0.9792619625384926)   
ACC&AUC   (0.4042617046818727, 0.9907439378796934)   
ACC&AUC  (0.21218487394957983, 0.9980615794561674)   

                                          2-Other  \
ACC&AUC   (0.285451197053407, 0.9897927762265037)   
ACC&AUC  (0.9815837937384899, 0.3560939161902691)   
ACC&AUC  (0.3247391037446286, 0.9355664986807091)   
ACC&AUC    (0.5494168201350522, 0.94519904911194)   
ACC&AUC  (0.2222222222222222, 0.9536883194030794)   

                                           3-Other  \
ACC&AUC   (0.3229166666666667, 0.9809465101419687)   
ACC&AUC   (0.31985294117647056, 0.936940612149858)   
ACC&AUC  (0.9865196078431373, 0.35522137939961207)   
ACC&AUC   (0.6176470588235294, 0.8561906743219154)   
ACC&AUC   (0.26654411

In [23]:
mean_acc_P, mean_acc_P_w, mean_acc_B, mean_acc_B_w = [], [], [], []
for i in range(5):
    tmp1, tmp2 = 0, 0
    for j in range(5):
        tmp1 += PIE_res[i][j][0]
        tmp2 += PIE_res_w[i][j][0]
    mean_acc_P.append(tmp1 / 5)
    mean_acc_P_w.append(tmp2 / 5)

for i in range(4):
    tmp1, tmp2 = 0, 0
    for j in range(4):
        tmp1 += Book_res[i][j][0]
        tmp2 += Book_res_w[i][j][0]
    mean_acc_B.append(tmp1 / 4)
    mean_acc_B_w.append(tmp2 / 4)

P_b_I = [[sum(mean_acc_P) / 5], [sum(mean_acc_P_w) / 5]]
B_b_I = [[sum(mean_acc_B) / 4], [sum(mean_acc_B_w) / 4]]

mean_acc_P_df = pd.DataFrame(mean_acc_P, columns=['mean_acc'], index=['1', '2', '3', '4', '5'])
mean_acc_P_df_w = pd.DataFrame(mean_acc_P_w, columns=['mean_acc'], index=['1', '2', '3', '4', '5'])
mean_acc_B_df = pd.DataFrame(mean_acc_B, columns=['mean_acc'], index=['1', '2', '3', '4'])
mean_acc_B_df_w = pd.DataFrame(mean_acc_B_w, columns=['mean_acc'], index=['1', '2', '3', '4'])
baseline_IWERM_P = pd.DataFrame(P_b_I, columns=['mean_acc'], index=['baseline', 'IWERM'])
baseline_IWERM_B = pd.DataFrame(B_b_I, columns=['mean_acc'], index=['baseline', 'IWERM'])

print('****************************************************************')
print(mean_acc_P_df)
print('****************************************************************')
print(mean_acc_P_df_w)
print('****************************************************************')
print(mean_acc_B_df)
print('****************************************************************')
print(mean_acc_B_df_w)
print('****************************************************************')
print(baseline_IWERM_P)
print('****************************************************************')
print(baseline_IWERM_B)
print('****************************************************************')

****************************************************************
   mean_acc
1  0.445027
2  0.453766
3  0.454167
4  0.588563
5  0.402184
****************************************************************
   mean_acc
1  0.913125
2  0.895567
3  0.883538
4  0.925279
5  0.853647
****************************************************************
   mean_acc
1   0.75300
2   0.78550
3   0.78175
4   0.81375
****************************************************************
   mean_acc
1  0.843250
2  0.882375
3  0.870500
4  0.883250
****************************************************************
          mean_acc
baseline  0.468742
IWERM     0.894231
****************************************************************
          mean_acc
baseline  0.783500
IWERM     0.869844
****************************************************************
